The goal of this notebook is to create a geodaframe containing information about each solar panel annotation. This dataframe will be useful in creating image mask pairs of tiles of the Cape Town geotiff images. The code to create the geodataframe and the image make pairs is available in this notebook.   

In [2]:
# import necessary packages
import json
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import rasterio
import ast
import cv2
import re
import shutil
import random
from rasterio.transform import rowcol
from rasterio.windows import Window
from shapely.geometry import Polygon
from shapely.geometry import shape
from shapely.geometry import box
from collections import namedtuple
from tqdm import tqdm
from PIL import Image

In [ ]:
# Determine the images that have been annotated
status = pd.read_excel(r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\CapeTown_ImageIDs.xlsx")
completed = status[status["Status"]=='Completed']
completed = completed[['Image ID', 'Annotator']]
completed

c:\Users\AICHA\miniconda3\envs\solar\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Image ID,Annotator
56,W07A_1,Gary Alvarez Mejia
63,W07A_16,Abby Finkle
72,W07A_24,Vanshika Mittal
106,W07C_10,Biz Yoder
107,W07C_11,Biz Yoder
...,...,...
2563,W57C_3,Fiona Bolte-Bradhurst
2564,W57C_4,Fiona Bolte-Bradhurst
2565,W57C_5,Fiona Bolte-Bradhurst
2568,W57C_8,Fiona Bolte-Bradhurst


There are 2807 images of Cape Town in total. 
As of Nov 29, there 544 completed images, including those that have no solar panel annotation. 151 geotiff images have annotations.
As of Fev 1st, there 731 completed images, including those that have no solar panel annotation. 251 geotiff images have annotations.

In [11]:
completed['Annotator'].unique()

array(['Gary Alvarez Mejia', 'Abby Finkle', 'Vanshika Mittal',
       'Biz Yoder', 'Zeinab Mukhtar', 'Shehr Naz Ashraf', 'Ye Khaung Oo',
       'Veena Shirsath', 'Brian Mulu Mutua', 'Halle Evans',
       'Ummamah Shah', 'Fiona Bolte-Bradhurst'], dtype=object)

Abby Finkle has only one completed image. Gary Alvarez Mejia has two.

I couldn't find Abby Finkle's annotations layer. Veena didn't upload an annotations layer.

In [ ]:
# Filter completed annotations
selected_annotators = ['Gary Alvarez Mejia', 'Abby Finkle', 'Vanshika Mittal',
       'Biz Yoder', 'Zeinab Mukhtar', 'Shehr Naz Ashraf', 'Ye Khaung Oo',
       'Veena Shirsath', 'Brian Mulu Mutua', 'Halle Evans',
       'Ummamah Shah', 'Fiona Bolte-Bradhurst']
filtered_completed_df = completed[completed['Annotator'].isin(selected_annotators)]

# Map annotator names to keys
key_mapping = {
    'Biz Yoder': 'biz',
    'Brian Mulu Mutua': 'mutua',
    'Fiona Bolte-Bradhurst': 'fiona',
    'Ummamah Shah': 'shah',
    'Ye Khaung Oo': 'ye',
    'Gary Alvarez Mejia': 'mejia',
    'Vanshika Mittal': 'mittal',
    'Zeinab Mukhtar': 'mukhtar',
    'Shehr Naz Ashraf': 'shehr',
    'Halle Evans': 'evans'
}

# Set file paths
file_paths = {
    'biz': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Biz Yoder\yoder_annotations 0128_v2.shp",
    'mutua': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Brian Mulu Mutua\mutua_annotations 1027.shp",
    'fiona': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Fiona Bolte-Bradhurst\1.31\bolte.bradhurst_annotations_1.31.shp",
    'shah': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Ummamah Shah\Shah_30.shp",
    'ye': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Ye Khaung Oo\Ye_annotations.shp",
    'mejia': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Gary Alvarez Mejia\11-13\alvarezmejia_annotations.shp",
    'mittal': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Vanshika Mittal\Mittal_annotations.shp",
    'mukhtar': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Zeinab Mukhtar\mostupdated\mukhtar_annotations_dec5.shp",
    'shehr': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Shehr Naz Ashraf\ashraf_annotations.shp",
    'evans': r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\Annotation layers\Halle Evans\evans_annotations_layer.shp"
}


annotator_image_ids = filtered_completed_df.groupby('Annotator')['Image ID'].apply(list).to_dict()
annotator_image_ids = {key_mapping[old_key]: value for old_key, value in annotator_image_ids.items()}

In [ ]:
# Extract the CRS of the images. This will be used to reproject the annotations layers to this CRS to avoid any inconsistencies.
input_tif = r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\AP2023_TIFFs_Bass\2023_RGB_8cm_W18B_5.tif"
with rasterio.open(input_tif) as dataset:
    geotif_crs = dataset.crs

# Clean the annotation layers be removing those marked as PV pool and PV heater
def clean_annotation_layers(layer):
    copy = layer
    copy = copy.to_crs(geotif_crs)
    copy['area'] = copy.geometry.area
    
    # remove annotations tagged as PV pool and PV heater
    if 'PV_Pool' in copy.columns:
        copy = copy[copy['PV_Pool'] != 1]

    if 'PV_pool' in copy.columns:
        copy = copy[copy['PV_pool'] != 1]

    if 'PV_heater' in copy.columns:
        copy = copy[copy['PV_heater'] != 1]
    
    columns_to_drop = ['layer', 'path', 'PV_heater', 'uncertflag', 'PV_Pool', 'PV_pool']
    copy = copy.drop([col for col in columns_to_drop if col in copy.columns], axis=1) 
    
    copy.reset_index(drop=True, inplace=True)       
    
    return copy

In [ ]:
# Load and process annotations
annotations_list = []
for annotator, path in file_paths.items():
    annotations = gpd.read_file(path)
    # print(annotations.columns)
    # print(annotator)
    annotations['annotator'] = annotator
    if (annotator != 'biz') & (annotator != 'evans'):
        annotations = annotations[annotations['path'].isnull()]
    annotations = clean_annotation_layers(annotations)
    annotations_list.append(annotations)

# Concatenate all annotations and add important polygon features (polygon's centroid coordinates)
annotations = pd.concat(annotations_list, ignore_index=True)
annotations.reset_index(drop=True, inplace=True)
annotations['id'] = annotations.index
annotations['centroid'] = annotations.geometry.centroid
annotations['centroid_latitude'] = annotations.centroid.y
annotations['centroid_longitude'] = annotations.centroid.x
annotations.drop(columns=['centroid'], inplace=True)
annotations = annotations[annotations['geometry'].notnull()]

# Add additional columns to annotations
annotations[['image_name', 'nw_corner_of_image_latitude', 'nw_corner_of_image_longitude', 
             'se_corner_of_image_latitude', 'se_corner_of_image_longitude']] = None

annotations

,id,area,geometry,annotator,centroid_latitude,centroid_longitude,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude
0,0,23.162657,"POLYGON ((-19993.55 -3769759.6, -19988.801 -37...",biz,-3.769763e+06,-19991.996891,None,None,None,None,None
1,1,8.567160,"POLYGON ((-13352.818 -3768899.929, -13345.695 ...",biz,-3.768902e+06,-13349.578806,None,None,None,None,None
2,2,9.287182,"POLYGON ((-13353.655 -3768901.389, -13346.512 ...",biz,-3.768904e+06,-13350.422909,None,None,None,None,None
3,3,8.580228,"POLYGON ((-13354.608 -3768903.024, -13347.583 ...",biz,-3.768906e+06,-13351.395241,None,None,None,None,None
4,4,10.808848,"POLYGON ((-13354.102 -3768907.616, -13353.441 ...",biz,-3.768910e+06,-13351.308980,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
19908,19908,4.039285,"POLYGON ((-33704.692 -3760339.71, -33701.724 -...",evans,-3.760340e+06,-33703.292947,None,None,None,None,None
19909,19909,2.332195,"POLYGON ((-33661.945 -3760112.949, -33660.812 ...",evans,-3.760114e+06,-33661.385134,None,None,None,None,None
19910,19910,26.211835,"POLYGON ((-30450.965 -3763339.573, -30447.33 -...",evans,-3.763343e+06,-30448.351613,None,None,None,None,None
19911,19911,25.601888,"POLYGON ((-30458.329 -3763342.933, -30454.279 ...",evans,-3.763346e+06,-30455.592527,None,None,None,None,None


Next, we need to determine which image each polygon belongs to. We are doing this by calculating the border coordinates for each annotator's images, then checking which polygons are within the images' bounds.

In [ ]:
def get_image_border_coordinates(image_path):
    with rasterio.open(image_path) as src:
        return src.bounds

folder_path = r'C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\AP2023_TIFFs_Bass'
prefix = '2023_RGB_8cm_'

# Dictionary to store border coordinates for each annotator's images
annotator_border_coordinates = {}
completed_ann = []

# Dictionary to store already processed images
processed_images = {}

# Iterate over each annotator and add the names of the images they completed
for annotator, image_names in annotator_image_ids.items():
    border_coordinates = {}
    for image_name in image_names:
        full_name = f"{prefix}{image_name}.tif"
        image_path = os.path.join(folder_path, full_name)
        
        if os.path.exists(image_path):
            if image_name not in processed_images:
                coordinates = get_image_border_coordinates(image_path)
                processed_images[image_name] = coordinates
            else:
                coordinates = processed_images[image_name]
            
            border_coordinates[image_name] = coordinates
    
    annotator_border_coordinates[annotator] = border_coordinates
    completed_ann.append(annotator)

In [ ]:
# some more data manipulations before associating the polygons to the images
BoundingBox = namedtuple('BoundingBox', ['left', 'bottom', 'right', 'top'])

transformed_dict = {}
for annotator, images in annotator_border_coordinates.items():
    transformed_dict[annotator] = {image: {'left': bounds.left, 'bottom': bounds.bottom, 'right': bounds.right, 'top': bounds.top} for image, bounds in images.items()}

annotator_border_coordinates = transformed_dict 

# Function to check if the centroid (consequently the polygon) is within a bounding box
def is_point_within_bounds(lat, lon, bounds):
    return bounds['left'] <= lon <= bounds['right'] and bounds['bottom'] <= lat <= bounds['top']

In [ ]:
flattened_coordinates = {}
for annotator, images in annotator_border_coordinates.items():
    flattened_coordinates.update(images)

# Iterate over each row in the annotations DataFrame
for idx, row in annotations.iterrows():
    # annotator = row['annotator']
    centroid_lat = row['centroid_latitude']
    centroid_lon = row['centroid_longitude']
    
    # Check which image the centroid belongs to
    for image_name, bounds in flattened_coordinates.items():
        if is_point_within_bounds(centroid_lat, centroid_lon, bounds):
            annotations.loc[idx, 'image_name'] = image_name
            annotations.loc[idx, 'nw_corner_of_image_latitude'] = bounds['top']
            annotations.loc[idx, 'nw_corner_of_image_longitude'] = bounds['left']
            annotations.loc[idx, 'se_corner_of_image_latitude'] = bounds['bottom']
            annotations.loc[idx, 'se_corner_of_image_longitude'] = bounds['right']
            break

# Now we associated the polygons to the images
annotations

,id,area,geometry,annotator,centroid_latitude,centroid_longitude,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude
0,0,23.162657,"POLYGON ((-19993.55 -3769759.6, -19988.801 -37...",biz,-3.769763e+06,-19991.996891,W16C_21,-3769000.0,-20000.0,-3770000.0,-19000.0
1,1,8.567160,"POLYGON ((-13352.818 -3768899.929, -13345.695 ...",biz,-3.768902e+06,-13349.578806,W16D_17,-3768000.0,-14000.0,-3769000.0,-13000.0
2,2,9.287182,"POLYGON ((-13353.655 -3768901.389, -13346.512 ...",biz,-3.768904e+06,-13350.422909,W16D_17,-3768000.0,-14000.0,-3769000.0,-13000.0
3,3,8.580228,"POLYGON ((-13354.608 -3768903.024, -13347.583 ...",biz,-3.768906e+06,-13351.395241,W16D_17,-3768000.0,-14000.0,-3769000.0,-13000.0
4,4,10.808848,"POLYGON ((-13354.102 -3768907.616, -13353.441 ...",biz,-3.768910e+06,-13351.308980,W16D_17,-3768000.0,-14000.0,-3769000.0,-13000.0
...,...,...,...,...,...,...,...,...,...,...,...
19908,19908,4.039285,"POLYGON ((-33704.692 -3760339.71, -33701.724 -...",evans,-3.760340e+06,-33703.292947,W36B_2,-3760000.0,-34000.0,-3761000.0,-33000.0
19909,19909,2.332195,"POLYGON ((-33661.945 -3760112.949, -33660.812 ...",evans,-3.760114e+06,-33661.385134,W36B_2,-3760000.0,-34000.0,-3761000.0,-33000.0
19910,19910,26.211835,"POLYGON ((-30450.965 -3763339.573, -30447.33 -...",evans,-3.763343e+06,-30448.351613,None,None,None,None,None
19911,19911,25.601888,"POLYGON ((-30458.329 -3763342.933, -30454.279 ...",evans,-3.763346e+06,-30455.592527,None,None,None,None,None


In [ ]:
# Saving the processed dataframe locally
# available_annotations = annotations[annotations['image_name'].notnull()]
# available_annotations.to_file(r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Teams\Team 1 Machine learning\CT - MachineLearning\S1 Machine Learning\available_annotations_feb.shp")

C:\Users\AICHA\AppData\Local\Temp\ipykernel_12604\1293296621.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  available_annotations.to_file(r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Teams\Team 1 Machine learning\CT - MachineLearning\S1 Machine Learning\available_annotations_feb.shp")
c:\Users\AICHA\miniconda3\envs\solar\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'centroid_latitude' to 'centroid_l'
  ogr_write(
c:\Users\AICHA\miniconda3\envs\solar\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'centroid_longitude' to 'centroid_1'
  ogr_write(
c:\Users\AICHA\miniconda3\envs\solar\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'nw_corner_of_image_latitude' to 'nw_corner_'
  ogr_write(
c:\Users\AICHA\miniconda3\envs\solar\lib\site-packages\pyogrio\raw.py:723: 

So far, we created a dataset that contains all of the available annotations, their geographical coordinates, each polygon centroid's coordinates, the annotator name, and the image each annotation belongs to. Now, we will calculate the pixel coordinates and the area in pixels of each polygon. These pixel corrdinates are necessary to cut the images into image-mask pairs.

In [ ]:
# This cell will run for a while. It calculates the pixel metrics of each polygon.
annotations = annotations[annotations['image_name'].notnull()]

def calculate_pixel_metrics_grouped(group, image_base_path):
    metrics = []
    for _, row in group.iterrows():
        geotiff_path = image_base_path + row['image_name'] + ".tif"
        try:
            with rasterio.open(geotiff_path) as src:
                transform = src.transform
                metric = calculate_pixel_metrics_dynamic(row, transform)
                metrics.append(metric)
        except FileNotFoundError:
            print(f"GeoTIFF not found: {geotiff_path}")
            metrics.append(None)
    
    return pd.DataFrame(metrics, index=group.index)

def calculate_pixel_metrics_dynamic(row, transform):
    polygon = shape(row['geometry'])
    centroid = polygon.centroid

    centroid_pixel_row, centroid_pixel_col = rowcol(transform, centroid.x, centroid.y)

    pixel_vertices = [
        rowcol(transform, vertex[0], vertex[1]) for vertex in polygon.exterior.coords
    ]

    pixel_polygon = shape({
        'type': 'Polygon',
        'coordinates': [[(c, r) for r, c in pixel_vertices]]
    })
    pixel_area = pixel_polygon.area

    return {
        'polygon_vertices_pixels': pixel_vertices,
        'centroid_latitude_pixels': centroid_pixel_row,
        'centroid_longitude_pixels': centroid_pixel_col,
        'area_pixels': pixel_area
    }

# Group the annotations by image_name
grouped_annotations = annotations.groupby('image_name')

image_base_path = r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\AP2023_TIFFs_Bass\2023_RGB_8cm_"
pixel_metrics_list = []

for name, group in tqdm(grouped_annotations, desc="Processing Images"):
    print(f"Processing image: {name}")
    metrics = calculate_pixel_metrics_grouped(group, image_base_path)
    pixel_metrics_list.append(metrics)

# Concatenate all metrics into a single DataFrame
pixel_metrics_df = pd.concat(pixel_metrics_list)

# Assign each metric to its respective column in the annotations DataFrame
annotations = annotations.join(pixel_metrics_df)

Processing Images:   0%|          | 0/184 [00:00<?, ?it/s]

Processing image: W07A_1


Processing Images:   1%|          | 1/184 [00:00<02:05,  1.46it/s]

Processing image: W07A_24


Processing Images:   1%|          | 2/184 [00:01<02:02,  1.49it/s]

...

Processing Images:  99%|█████████▉| 182/184 [4:40:57<03:34, 107.10s/it]

Processing image: W57C_8


Processing Images:  99%|█████████▉| 183/184 [4:42:40<01:45, 105.91s/it]

Processing image: W57C_9


Processing Images: 100%|██████████| 184/184 [4:44:31<00:00, 92.78s/it] 


In [ ]:
# Check the order of the columns of the annotations dataframe and replace them here
annotations.columns = ['id', 'annotator', 'area', 'centroid_latitude', 'centroid_longitude',
       'image_name', 'nw_corner_of_image_latitude',
       'nw_corner_of_image_longitude', 'se_corner_of_image_latitude',
       'se_corner_of_image_longitude', 'polygon_vertices_pixels',
       'centroid_latitude_pixels', 'centroid_longitude_pixels', 'area_pixels',
       'geometry']

# Save the final dataframe. Now we are ready to create the image-mask pairs
annotations.to_file(r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Teams\Team 1 Machine learning\CT - MachineLearning\S1 Machine Learning\annotations_final_feb.geojson")

Some necessary data manipulation of the annotations dataframe's polygon_vertices_pixels column before creating image mask pairs.

In [105]:
df = annotations.copy()

# Clean the strings to remove np.int32 calls and fix any incomplete strings
df['polygon_vertices_pixels'] = df['polygon_vertices_pixels'].apply(
    lambda x: re.sub(r'np\.int32\((\d+)\)', r'\1', x)
)

# Replace parentheses with square brackets
df['polygon_vertices_pixels'] = df['polygon_vertices_pixels'].apply(
    lambda x: re.sub(r'\((\d+), (\d+)\)', r'[\1, \2]', x)
)

# Ensure all tuples are properly closed
def fix_incomplete_tuples(polygon_str):
    # Find all tuples
    tuples = re.findall(r'\[\d+, \d+\]', polygon_str)
    # Join them back into a string
    fixed_str = '[' + ', '.join(tuples) + ']'
    return fixed_str

df['polygon_vertices_pixels'] = df['polygon_vertices_pixels'].apply(fix_incomplete_tuples)

# Convert the string representation of lists back to actual lists
df['polygon_vertices_pixels'] = df['polygon_vertices_pixels'].apply(
    lambda x: ast.literal_eval(x)
)

print(df['polygon_vertices_pixels'])

0        [[9495, 80], [9522, 139], [9575, 118], [9545, ...
1        [[11249, 8089], [11302, 8178], [11313, 8172], ...
2        [[11267, 8079], [11321, 8168], [11333, 8160], ...
3        [[11287, 8067], [11341, 8155], [11351, 8147], ...
4        [[11345, 8073], [11332, 8081], [11367, 8137], ...
                               ...                        
11303    [[6492, 3431], [6500, 3479], [6523, 3474], [65...
11304    [[8268, 1976], [8264, 1990], [8280, 1996], [82...
11305    [[7834, 2180], [7825, 2200], [7854, 2211], [78...
11306    [[4246, 3691], [4249, 3728], [4265, 3727], [42...
11307    [[1411, 4225], [1411, 4239], [1437, 4239], [14...
Name: polygon_vertices_pixels, Length: 11308, dtype: object


In [106]:
annotations = df

Now that the dataframe is ready, we can finally create the image mask pairs. Below are the functions we use to do so

In [ ]:
def create_mask(image_shape, polygons):
    mask = np.zeros(image_shape[:2], dtype="uint8")
    for polygon in polygons:
        cv2.fillPoly(mask, [polygon], 255)
    flipped_mask = cv2.flip(mask, 0)
    rotated_mask = cv2.rotate(flipped_mask, cv2.ROTATE_90_CLOCKWISE)
    return rotated_mask

def save_tile_and_mask(tile, mask, tile_index_pixels, tile_dir, mask_dir, image_name):
    tile_filename = os.path.join(tile_dir, f'i_{image_name}_{tile_index_pixels}.png')
    mask_filename = os.path.join(mask_dir, f'm_{image_name}_{tile_index_pixels}.png')
    cv2.imwrite(tile_filename, cv2.cvtColor(tile, cv2.COLOR_RGB2BGR))
    cv2.imwrite(mask_filename, mask)

def adjust_polygon_coordinates(polygons, x_offset, y_offset):
    adjusted_polygons = []
    for polygon in polygons:
        adjusted_polygon = polygon - np.array([x_offset, y_offset])
        adjusted_polygons.append(adjusted_polygon)
    return adjusted_polygons

def process_geotiff(image_name, geotiff_path, tile_size, df, tile_dir, mask_dir):
    with rasterio.open(geotiff_path) as src:
        geotiff_array = src.read()

        if len(geotiff_array.shape) == 3:
            geotiff_array = np.transpose(geotiff_array, (1, 2, 0))

        height, width = geotiff_array.shape[:2]

        # Calculate padding
        pad_height = (tile_size - height % tile_size) % tile_size
        pad_width = (tile_size - width % tile_size) % tile_size

        # Add padding to the image
        padded_image = np.pad(geotiff_array, ((0, pad_height), (0, pad_width), (0, 0)), mode='constant')

        padded_height, padded_width = padded_image.shape[:2]

        tile_index = 0
        for y in range(0, padded_height, tile_size):
            for x in range(0, padded_width, tile_size):
                tile = padded_image[y:y+tile_size, x:x+tile_size]

                polygons_in_tile = []
                for _, row in df.iterrows():
                    polygon = row['polygon_vertices_pixels']
                    
                    bounds = {
                        'left': x,
                        'right': x+tile_size,
                        'bottom': y+tile_size,
                        'top': y
                    }
                    
                    if (bounds['left'] <= row['centroid_longitude_pixels'] <= bounds['right'] and bounds['top'] <= row['centroid_latitude_pixels'] <= bounds['bottom']):
                        polygons_in_tile.append(polygon)
                    
                adjusted_polygons = adjust_polygon_coordinates(polygons_in_tile, y, x)

                mask = create_mask(tile.shape, adjusted_polygons)
                
                # save only the masks and tiles that contain annotations
                if np.any(mask > 0):
                    tile_index_pixels = str(int(y/tile_size)) + "_" + str(int(x/tile_size))
                    save_tile_and_mask(tile, mask, tile_index_pixels, tile_dir, mask_dir, image_name)
                
                tile_index += 1


def process_all_images_in_folder(folder_path, annotations_df, tile_size, tile_dir, mask_dir, processed_images_list):
    unique_images = annotations_df['image_name'].unique()
    idx = 0
    for image_name in unique_images:
        image_path = folder_path + image_name + ".tif"
        
        if os.path.exists(image_path):
            print(image_name)
            print(idx)
            idx += 1
            processed_images_list.append(image_name)
            
            image_annotations_df = annotations_df[annotations_df['image_name'] == image_name]
            process_geotiff(image_name, image_path, tile_size, image_annotations_df, tile_dir, mask_dir)


In [ ]:
# path where you want to save the tiles
tile_dir = r'C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Teams\Team 1 Machine learning\CT - MachineLearning\S1 Machine Learning\dataset\tiles_320'
# path where you want to save the images
mask_dir = r'C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Teams\Team 1 Machine learning\CT - MachineLearning\S1 Machine Learning\dataset\masks_320'
os.makedirs(tile_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# path where the original images are saved
dataset_path = r"C:\Users\AICHA\Box\Cape Town Energy Transitions Bass Connections\Class Materials 2024-2025\Aerial Imagery\AP2023_TIFFs_Bass\2023_RGB_8cm_"

# Choose the tile size of your new dataset
tile_size = 320 

# The names of the processed images will be printed to monitor the progress
processed_images_list = []
process_all_images_in_folder(dataset_path, annotations, tile_size=320, tile_dir=tile_dir, mask_dir=mask_dir, processed_images_list=processed_images_list)

W16C_21
W16D_17
W18B_8
W18B_9
W48C_11
W57C_21
W57C_17
W57C_24
W57B_10
W36A_2
W57B_5
W57B_13
W57B_14
W45C_16
W25A_3
W57C_25
W57C_23
W57C_8
W57C_9
W57C_12
W57A_18
W57A_19
W57A_23
W57A_24
W57A_22
W57C_10
W57C_3
W57C_4
W57C_2
W57A_21
W47C_1
W36A_8
W36A_3
W16C_22
W25A_6
W18B_5
W47C_2
W57A_25
W57C_5
W25C_12
W25C_13
W25C_11
W47C_6
W57B_12
W16D_25
W16D_24
W07C_10
W07C_11
W07C_12
W07C_13
W07C_16
W07C_17
W07C_2
W07C_21
W07C_22
W07C_23
W07C_3
W07C_4
W07C_5
W07C_6
W07C_7
W07C_8
W07C_9
W07D_1
W07D_6
W08A_1
W08A_12
W08A_2
W08B_4
W08B_9
W12A_17
W12A_21
W12A_22
W12A_23
W12C_11
W12C_12
W12C_16
W12C_4
W12C_6
W13C_11
W13C_13
W13C_16
W13C_18
W13C_22
W13C_8
W14A_11
W14A_12
W14A_21
W16C_14
W16C_15
W16C_17
W16C_18
W16C_19
W16C_20
W16D_12
W16D_16
W16D_21
W16D_22
W16D_23
W16D_7
W17B_11
W33A_9
W33A_10
W33A_18
W33B_2
W33C_7
W33C_8
W33C_9
W33C_10
W57A_17
W57B_2
W57B_3
W57B_4
W57B_6
W57B_7
W57B_8
W57B_9
W57B_18
W57B_19
W57B_20
W57B_15
W57C_22
W48C_6
W48C_16
W48C_22
W49A_11
W49A_2
W49A_16
W50D_4
W50D_5
W50D_25
W47C

There are a lot of images that don't contain any solar panel. To avoid wasting training resources, we'll only select the images that contain the panels to work with (it may still be useful to train the model with the images that don't contain any panels to increase model robustness)

In [ ]:
# Organizing the folders locally -> copy the masks and tiles to the destination folders

# copy the masks that contain the target to a seperate folder
def check_mask_has_target(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    return np.any(mask > 0)

def copy_masks_with_target(source_folder, destination_folder):
    os.makedirs(destination_folder, exist_ok=True)
    
    for filename in os.listdir(source_folder):
        file_path = os.path.join(source_folder, filename)
        
        if check_mask_has_target(file_path):
            shutil.copy(file_path, destination_folder)
            print(f"Copied {filename} to {destination_folder}")
            
# copy the corresponding images to a new folder
def copy_corresponding_images(mask_folder, image_folder, destination_folder):
    os.makedirs(destination_folder, exist_ok=True)
    
    for mask_filename in os.listdir(mask_folder):
        image_filename = 'i' + mask_filename[1:]
        image_path = os.path.join(image_folder, image_filename)
        
        if os.path.exists(image_path):
            shutil.copy(image_path, destination_folder)
            print(f"Copied {image_filename} to {destination_folder}")



In [ ]:
source_folder = '/home/as1233/data/cape_town/masks'
destination_folder = '/home/as1233/data/cape_town/masks_target'

copy_masks_with_target(source_folder, destination_folder)

mask_folder = '/home/as1233/data/cape_town/masks_target'
image_folder = '/home/as1233/data/cape_town/tiles'
destination_folder = '/home/as1233/data/cape_town/images_target'

copy_corresponding_images(mask_folder, image_folder, destination_folder)

In [ ]:
# split the dataset into train, test, and val datasets

def split_data(mask_folder, image_folder, train_folder, val_folder, test_folder, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    os.makedirs(os.path.join(train_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(train_folder, 'masks'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(val_folder, 'masks'), exist_ok=True)
    os.makedirs(os.path.join(test_folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(test_folder, 'masks'), exist_ok=True)
    
    mask_files = os.listdir(mask_folder)
    
    random.shuffle(mask_files)
    
    total_files = len(mask_files)
    train_count = int(total_files * train_ratio)
    val_count = int(total_files * val_ratio)
    test_count = total_files - train_count - val_count
    
    train_files = mask_files[:train_count]
    val_files = mask_files[train_count:train_count + val_count]
    test_files = mask_files[train_count + val_count:]
    
    def copy_files(file_list, dest_image_folder, dest_mask_folder):
        for mask_filename in file_list:
            shutil.copy(os.path.join(mask_folder, mask_filename), dest_mask_folder)
            
            image_filename = 'i' + mask_filename[1:]
            shutil.copy(os.path.join(image_folder, image_filename), dest_image_folder)
    
    copy_files(train_files, os.path.join(train_folder, 'images'), os.path.join(train_folder, 'masks'))
    copy_files(val_files, os.path.join(val_folder, 'images'), os.path.join(val_folder, 'masks'))
    copy_files(test_files, os.path.join(test_folder, 'images'), os.path.join(test_folder, 'masks'))

mask_folder = '/home/as1233/data/cape_town/masks_target'
image_folder = '/home/as1233/data/cape_town/images_target'
train_folder = '/home/as1233/data/cape_town/train'
val_folder = '/home/as1233/data/cape_town/val'
test_folder = '/home/as1233/data/cape_town/test'

split_data(mask_folder, image_folder, train_folder, val_folder, test_folder)